In [ ]:
import numpy as np

import matplotlib.cm as cm
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Simulation settings (1.7; 300; 0.1)
w = 1.7 # Omega
domain_a = (300, 300) # Y,X domain_a

# Walls
wall = np.array([1, 1, 1, 1]) # Wall existence (E, N, W, S)
wall_speed = np.array([0, 0.1, 0, 0]) # Wall movement speed (E, N, W, S)

# Reynolds Number
print((wall_speed[1] * domain_a[1]) / (1/3 * (1/w - 1/2)))

In [ ]:
# Constant arrays
w_c = np.array([4/9] + [1/9] * 4 + [1/36] * 4)
c_ac  = np.array([[0, 1, 0, -1, 0, 1, -1, -1, 1], [0, 0, 1, 0, -1, 1, 1, -1, -1]])
wall_c = np.array([[8, 1, 5], [5, 2, 6], [6, 3, 7], [7, 4, 8]]) # Walls with corresponding channels (E, N, W, S)
wall_f = np.array([-1/6, 0, +1/6]) # Wall speed factors
wall_p = np.array([[1, 2, 1, 0, 1, 2, 0, 0, 2], [1, 1, 2, 1, 0, 2, 2, 0, 0], [1, 0, 1, 2, 1, 0, 2, 2, 0], [1, 1, 0, 1, 2, 0, 0, 2, 2]]) # Wall Probability Weights (E, N, W, S)


# Stream the lattice channels, apply boundary conditions
def stream(lattice_yxc):
    # Calculate Wall Reflection
    if wall[0]: # East
        reflectE = lattice_yxc[ :,-1, wall_c[0]] - wall_speed[0] * np.einsum('yc, c, f -> yf', lattice_yxc[:, -1, :], wall_p[0], wall_f)
    if wall[1]: # North
        reflectN = lattice_yxc[ 0, :, wall_c[1]].T + wall_speed[1] * np.einsum('xc, c, f -> xf', lattice_yxc[0, :, :], wall_p[1], wall_f)
    if wall[2]: # West
        reflectW = lattice_yxc[ :, 0, wall_c[2]] + wall_speed[2] * np.einsum('yc, c, f -> yf', lattice_yxc[:, 0, :], wall_p[2], wall_f)
    if wall[3]: # South
        reflectS = lattice_yxc[-1, :, wall_c[3]].T - wall_speed[3] * np.einsum('xc, c, f -> xf', lattice_yxc[-1, :, :], wall_p[3], wall_f)

    # Stream Channels
    lattice_yxc[:, :, 1] = np.roll(lattice_yxc[:, :, 1], 1, axis=1) # Channel 1 [0, +1]
    lattice_yxc[:, :, 2] = np.roll(lattice_yxc[:, :, 2], -1, axis=0) # Channel 2 [-1, 0]
    lattice_yxc[:, :, 3] = np.roll(lattice_yxc[:, :, 3], -1, axis=1) # Channel 3 [0, -1]
    lattice_yxc[:, :, 4] = np.roll(lattice_yxc[:, :, 4], 1, axis=0) # Channel 4 [+1, 0]

    lattice_yxc[:, :, 5] = np.roll(lattice_yxc[:, :, 5], [-1, 1], axis=(0, 1)) # Channel 5 [-1, +1]
    lattice_yxc[:, :, 6] = np.roll(lattice_yxc[:, :, 6], [-1, -1], axis=(0, 1)) # Channel 6 [-1, -1]
    lattice_yxc[:, :, 7] = np.roll(lattice_yxc[:, :, 7], [1, -1], axis=(0, 1)) # Channel 7 [+1, -1]
    lattice_yxc[:, :, 8] = np.roll(lattice_yxc[:, :, 8], [1, 1], axis=(0, 1)) # Channel 8 [+1, +1]

    # Apply Wall Reflection
    if wall[0]: # East
        lattice_yxc[ :,-1, wall_c[2]] = reflectE
    if wall[2]: # West
        lattice_yxc[ :, 0, wall_c[0]] = reflectW
    if wall[3]: # South
        lattice_yxc[-1, :, wall_c[1]] = reflectS.T
    if wall[1]: # North
        lattice_yxc[ 0, :, wall_c[3]] = reflectN.T


# Calculate the physical properties (probability density values, velocitys) of the lattice
def physics(lattice_yxc):
    p_yx = np.einsum('yxc -> yx', lattice_yxc) # Calculate Probability Density
    v_yxa = np.einsum('yxc, ac, yx -> yxa', lattice_yxc, c_ac, 1/p_yx) # Calculate XY Speed

    return p_yx, v_yxa


# Calculate the equilibrium values of the lattice
def equilibrium(p_yx, v_yxa):
    # Splitted Equilibrium Formula
    feq_yxc = np.einsum('ac, yxa -> yxc', c_ac, v_yxa)
    feq_yxc = 1 + 3 * feq_yxc + 9/2 * np.square(feq_yxc) - 3/2 * np.sum(np.square(v_yxa), axis=2)[:, :, np.newaxis]
    feq_yxc = np.einsum('c, yx, yxc -> yxc', w_c, p_yx, feq_yxc)

    return feq_yxc


# Update lattice according to the choosen viscosity and the calculated equilibrium values
def collision(lattice_yxc, poiseuille=False):
    # Flow Out Right, Poiseuille Flow
    if poiseuille:
        lattice_yxc[:, -1, wall_c[2]] = lattice_yxc[:, -2, wall_c[2]]

    p_yx, v_yxa = physics(lattice_yxc)

    # Apply static inlet
    if poiseuille:
        #p_yx = np.where(np.isnan(inletP_yx), p_yx, inletP_yx)
        v_yxa = np.where(np.isnan(inletV_yxa), v_yxa, inletV_yxa)

    feq_yxc = equilibrium(p_yx, v_yxa)

    if poiseuille:
        lattice_yxc = np.where(np.isnan(inletP_yx[:, :, np.newaxis]), lattice_yxc, feq_yxc)

    # Update Step
    lattice_yxc += w * (feq_yxc - lattice_yxc)

    return p_yx, v_yxa, lattice_yxc


# Initialize Lattice
def initialize(setup=0):
    p_yx = np.ones(domain_a)
    v_yxa = np.zeros(domain_a + (2,))

    if setup == 1: # 2.1 / 4.4
        #p_yx[domain_a[0]//2, domain_a[1]//2] = 1.5
        p_yx[40, 90] = 1.5

    if setup == 2: # 2.1, Density, Sinus Y
        dist_x = 0.5 * np.sin(2 * np.pi * np.arange(domain_a[1]) / domain_a[1])
        p_yx = np.ones(domain_a) + dist_x

    if setup == 3: # 2.2 Shear Wave, Sinus X
        dist_y = 0.5 * np.sin(2 * np.pi * np.arange(domain_a[0]) / domain_a[0])
        v_yxa[:, :, 0] = dist_y[:, np.newaxis]

    if setup == 4: # 4.3 Poiseuille Flow
        inletP_yx[:, 0] = 1
        inletV_yxa[:, 0, :] = [0.2, 0]

        p_yx = np.where(np.isnan(inletP_yx), 1, inletP_yx)
        v_yxa = np.where(np.isnan(inletV_yxa), 0, inletV_yxa)

    return equilibrium(p_yx, v_yxa)

In [ ]:
# Execute Simulation
def run(lattice_yxc):
    x_i, y_i = np.linspace(0, domain_a[1], domain_a[1]), np.linspace(0, domain_a[0], domain_a[0]) # Visualization Coordinates

    for i in range(1000001):
        stream(lattice_yxc)
        p_yx, v_yxa, lattice_yxc = collision(lattice_yxc)

        # Visualize
        if True and i % 1000 == 0:
            print(i, np.sum(p_yx))
            plt.figure(figsize=(10, int((domain_a[1] / domain_a[0]) * 10)))
            canvas = plt.subplot()
            canvas.set_xlim([x_i[0], x_i[-2]])
            canvas.set_ylim([y_i[-2], y_i[0]])
            canvas.imshow(p_yx, vmin=0.5, vmax=1.5)
            canvas.streamplot(x_i, y_i, v_yxa[:,:,0], -v_yxa[:,:,1], color='black')
            canvas.get_xaxis().set_visible(False)
            canvas.get_yaxis().set_visible(False)
            plt.savefig('lid_drive_cavity-{}.png'.format(i), dpi=300)
            plt.show()

lattice_yxc = initialize(0)
run(lattice_yxc)

In [ ]:
# Density Diffusion
w = 1.2
domain_a = (50, 100)

wall = np.array([1, 1, 1, 1]) # Wall existence (E, N, W, S)
wall_speed = np.array([0, 0, 0, 0]) # Wall movement speed (E, N, W, S)

def run(lattice_yxc):
    x_i, y_i = np.linspace(0, domain_a[1], domain_a[1]), np.linspace(0, domain_a[0], domain_a[0]) # Visualization Coordinates

    for i in range(100):
        stream(lattice_yxc)
        p_yx, v_yxa, lattice_yxc = collision(lattice_yxc)

        # Visualize
        if True and i % 10 == 0:
            print(i, np.sum(p_yx))
            plt.figure()
            canvas = plt.subplot()
            canvas.set_xlim([x_i[0], x_i[-2]])
            canvas.set_ylim([y_i[-2], y_i[0]])
            canvas.imshow(p_yx, vmin=0.5, vmax=1.5)
            canvas.streamplot(x_i, y_i, v_yxa[:,:,0], -v_yxa[:,:,1], color='black')
            canvas.get_xaxis().set_visible(False)
            canvas.get_yaxis().set_visible(False)
            plt.savefig('density_diffusion-{}.png'.format(i), dpi=300)
            plt.show()

lattice_yxc = initialize(1)
run(lattice_yxc)

In [ ]:
# Setup: Density Shear Wave Decay
w = 1.2
domain_a = (50, 100)

wall = np.array([0, 0, 0, 0]) # Wall existence (E, N, W, S)
wall_speed = np.array([0, 0, 0, 0]) # Wall movement speed (E, N, W, S)

def run(lattice_yxc):
    x_i, y_i = np.linspace(0, domain_a[1], domain_a[1]), np.linspace(0, domain_a[0], domain_a[0]) # Visualization Coordinates

    for i in range(100):
        stream(lattice_yxc)
        p_yx, v_yxa, lattice_yxc = collision(lattice_yxc)

        # Visualize
        if True and i % 10 == 0:
            print(i, np.sum(p_yx))
            plt.figure()
            canvas = plt.subplot()
            if False:
                canvas.set_xlim([x_i[0], x_i[-2]])
                canvas.set_ylim([y_i[-2], y_i[0]])
                canvas.imshow(p_yx, vmin=0.5, vmax=1.5)
                canvas.streamplot(x_i, y_i, v_yxa[:,:,0], -v_yxa[:,:,1], color='black')
                canvas.get_xaxis().set_visible(False)
                canvas.get_yaxis().set_visible(False)
                plt.savefig('density_shear_diffusion-{}.png'.format(i), dpi=300)
            else:
                canvas.set_ylim([0.5, 1.5])
                canvas.axhline(y=1.0, color='black')
                canvas.plot(np.arange(domain_a[1]), p_yx[domain_a[0]//2, :], color='r', label='density, longitudinal section') # Density Profile
                canvas.get_xaxis().set_visible(False)
                plt.ylabel('density')
                plt.legend()
                plt.savefig('density_shear_diffusion_profile-{}.png'.format(i), dpi=300)
            plt.show()

lattice_yxc = initialize(2)
run(lattice_yxc)

In [ ]:
# Setup: Shear Wave Decay
domain_a = (100, 100)

wall = np.array([0, 0, 0, 0]) # Wall existence (E, N, W, S)
wall_speed = np.array([0, 0, 0, 0]) # Wall movement speed (E, N, W, S)

# Viscosity Evaluation
def run(lattice_yxc):
    speed_history = []

    for i in range(10000):
        stream(lattice_yxc)
        p_yx, v_yxa, lattice_yxc = collision(lattice_yxc)

        # Visualize
        if True and i % 100 == 0 and w == 1.2:
            print(i, np.sum(p_yx))
            plt.figure()
            canvas = plt.subplot()
            canvas.set_xlim([-0.5, 0.5])
            canvas.plot(v_yxa[:, domain_a[1]//2, 0], np.arange(domain_a[0])[::-1], color='r', label='x-velocity, cross section') # Velocity Profile
            canvas.get_yaxis().set_visible(False)
            plt.xlabel('x-velocity')
            plt.legend()
            plt.savefig('shear_wave_decay_profile-{}-{}.png'.format(w, i), dpi=300)
            plt.show()

        # Viscosity Calculation
        speed_history.append(np.max(v_yxa[:, :, 0]))

    return speed_history

viscosity = []
for w in np.around(np.arange(0.2, 1.8, 0.1), 1):
    lattice_yxc = initialize(3)
    speed_history = run(lattice_yxc)

    x_values = np.arange(len(speed_history))
    k_square = np.square(2 * np.pi / domain_a[0])

    # Plotting Speed to Time
    print('omega', w)
    y_values = 0.5 * np.exp(- 1/3 * (1/w - 1/2) * k_square * x_values)
    plt.plot(x_values, y_values, label='Analytical speed')
    plt.plot(x_values, speed_history, linestyle='--', label='Measured speed')
    plt.xlabel('Timestep')
    plt.ylabel('Maximum speed')
    plt.legend()
    plt.savefig('shear_wave_decay-{}.png'.format(w), dpi=300)
    plt.show()

    # Calculate Simulated Viscosity
    v_simulated = 0.0
    v_error = np.inf
    step_domain_a = 0.001

    while True: # Fit exponential formula to historic values
        speed = 0.5 * np.exp(-v_simulated * k_square * x_values[1:])
        error = np.mean(np.square(np.array(speed_history[1:]) - speed))
        if v_error >= error:
            v_simulated += step_domain_a
            v_error = error
        else:
            v_simulated -= step_domain_a
            break

    viscosity.append([w, 1/3 * (1/w - 1/2), v_simulated])

# Viscosity (Simulated, Analytical) to Omega
viscosity = np.array(viscosity)
plt.plot(viscosity[:, 0], viscosity[:, 1], label='Analytical viscosity')
plt.plot(viscosity[:, 0], viscosity[:, 2], linestyle='--', label='Measured viscosity')
plt.xlabel('Omega')
plt.ylabel('Viscosity')
plt.legend()
plt.savefig('shear_wave_decay.png', dpi=300)
plt.show()

In [ ]:
# Setup: Poiseuille Flow
w = 1.2
domain_a = (50, 100) # Y,X domain_a

wall = np.array([0, 1, 0, 1]) # Wall existence (E, N, W, S)
wall_speed = np.array([0, 0, 0, 0]) # Wall movement speed (E, N, W, S)

# Static Inlet
#inletP_yx = np.full(domain_a, np.nan) # Static Roh inlet
inletV_yxa = np.full(domain_a + (2,), np.nan) # Static speed inlet

# Velocity Evaluation
max_step = (2001, 10)
def run(lattice_yxc):
    x_i, y_i = np.linspace(0, domain_a[1], domain_a[1]), np.linspace(0, domain_a[0], domain_a[0]) # Visualization Coordinates

    for i in range(max_step[0]):
        stream(lattice_yxc)
        p_yx, v_yxa, lattice_yxc = collision(lattice_yxc, poiseuille=True)

        # Visualize
        if True and (i % max_step[1] == 0):
            print(i, np.sum(p_yx))

            plt.figure()
            canvas = plt.subplot()
            if False:
                canvas.set_xlim([x_i[0], x_i[-2]])
                canvas.set_ylim([y_i[-2], y_i[0]])
                #canvas.imshow(p_yx, vmin=0.5, vmax=1.5)
                canvas.streamplot(x_i, y_i, v_yxa[:,:,0], -v_yxa[:,:,1], color='black')
                canvas.get_xaxis().set_visible(False)
                canvas.get_yaxis().set_visible(False)
                plt.savefig('poiseuille_flow-{}.png'.format(i), dpi=300)
            else:
                if True: # Velocity Profile X
                    canvas.set_ylim([0, 0.3])
                    canvas.plot(np.arange(domain_a[1]), v_yxa[domain_a[0]//2, :, 0], color='r', label='x-velocity, longitudinal section')
                    canvas.get_xaxis().set_visible(False)
                    plt.ylabel('x-velocity')
                    plt.legend()
                    plt.savefig('poiseuille_flow_profileX-{}.png'.format(i), dpi=300)
                else: # Velocity Profile Y
                    y = np.arange(domain_a[0])[::-1]
                    canvas.set_xlim([0, 0.25])
                    canvas.plot(v_yxa[:, domain_a[1]//2, 0], y, linestyle='--', label='Measured speed')
                    canvas.get_yaxis().set_visible(False)
                    plt.legend()
                    plt.savefig('poiseuille_flow_profileY-{}.png'.format(i), dpi=300)
            plt.show()

    # Velocity Profile
    return v_yxa[:, domain_a[1]//2, 0]

lattice_yxc = initialize(4)
print('Top Speed:', run(lattice_yxc)[domain_a[0]//2])

In [ ]:
# Setup: Couette Flow
w = 1.2
domain_a = (50, 50)

wall = np.array([0, 1, 0, 1]) # Wall existence (E, N, W, S)
wall_speed = np.array([0, 0.2, 0, 0]) # Wall movement speed (E, N, W, S)

# Velocity Evaluation
max_step = (10001, 1000)
def run(lattice_yxc):
    x_i, y_i = np.linspace(0, domain_a[1], domain_a[1]), np.linspace(0, domain_a[0], domain_a[0]) # Visualization Coordinates

    for i in range(max_step[0]):
        stream(lattice_yxc)
        p_yx, v_yxa, lattice_yxc = collision(lattice_yxc)

        # Visualize
        if True and (i % max_step[1] == 0):
            print(i, np.sum(p_yx))

            plt.figure()
            canvas = plt.subplot()
            if False:
                canvas.set_xlim([x_i[0], x_i[-2]])
                canvas.set_ylim([y_i[-2], y_i[0]])
                #canvas.imshow(p_yx, vmin=0.5, vmax=1.5)
                canvas.streamplot(x_i, y_i, v_yxa[:,:,0], -v_yxa[:,:,1], color='black')
                canvas.get_xaxis().set_visible(False)
                canvas.get_yaxis().set_visible(False)
                plt.savefig('couette_flow-{}.png'.format(i), dpi=300)
            else:
                canvas.axhline(y=domain_a[0], color='black')
                canvas.axhline(y=-1.0, color='black')
                canvas.plot(v_yxa[:, domain_a[1]//2, 0], np.arange(domain_a[0])[::-1], color='r', label='x-velocity, cross section') # Velocity Profile
                canvas.get_yaxis().set_visible(False)
                plt.xlabel('x-velocity')
                plt.legend()
                plt.savefig('couette_flow_profile-{}.png'.format(i), dpi=300)
            plt.show()

    # Velocity Profile
    return v_yxa[:, domain_a[1]//2, 0]

lattice_yxc = initialize()
canal_speed = run(lattice_yxc)

print('Top Speed:', canal_speed[0])